In [68]:
pip install cartopy


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


路線図グラフ作成

In [102]:
import pandas as pd
import networkx as nx
import folium
import random
import osmnx as ox


# --- データ読み込み ---
station = pd.read_csv("/workspace/駅データまとめ/station20250523free.csv", dtype=str)
join = pd.read_csv("/workspace/駅データまとめ/join20250430.csv", dtype=str)
line = pd.read_csv("/workspace/駅データまとめ/line20250523free.csv", dtype=str)

# --- 横浜市内の駅のみ抽出 ---
station_yokohama = station[station['address'].str.contains('横浜', na=False)].copy()
station_yokohama['lat'] = station_yokohama['lat'].astype(float)
station_yokohama['lon'] = station_yokohama['lon'].astype(float)
yokohama_station_codes = set(station_yokohama['station_cd'])

# --- グラフ構築 ---
G = nx.Graph()
station_pos = station_yokohama.set_index('station_cd')[['lat', 'lon']].to_dict('index')

# ノード追加
for _, row in station_yokohama.iterrows():
    G.add_node(row['station_cd'], name=row['station_name'], pos=(row['lat'], row['lon']))

# エッジ追加（距離を重み）
edges = join[
    (join['station_cd1'].isin(yokohama_station_codes)) &
    (join['station_cd2'].isin(yokohama_station_codes))
].copy()

for _, row in edges.iterrows():
    cd1, cd2 = row['station_cd1'], row['station_cd2']
    if cd1 in station_pos and cd2 in station_pos:
        lat1, lon1 = station_pos[cd1]['lat'], station_pos[cd1]['lon']
        lat2, lon2 = station_pos[cd2]['lat'], station_pos[cd2]['lon']
        dist = ((lat1 - lat2)**2 + (lon1 - lon2)**2) ** 0.5
        G.add_edge(cd1, cd2, weight=dist)

# 同名駅を重み0で接続（乗換）
for name, group in station_yokohama.groupby('station_name'):
    codes = list(group['station_cd'])
    for i in range(len(codes)):
        for j in range(i + 1, len(codes)):
            G.add_edge(codes[i], codes[j], weight=0)


徒歩ルート生成

In [90]:
G_walk = ox.graph_from_place('Yokohama, Japan', network_type='walk')

In [140]:
# 無向グラフに変換
G_walk = G_walk.to_undirected(reciprocal=False)

In [138]:
# みなとみらい中心（緯度, 経度）
center_point = (35.4553642, 139.6296088)

# 半径（メートル単位）例：1000m（1km）
distance = 2000

G_drive = ox.graph_from_point(center_point, dist=distance, network_type='drive')

In [161]:
# --- 観光スポットデータ ---
spots = [
    {"name": "横浜中華街", "lat": 35.4432993, "lon": 139.6456246},
    {"name": "赤レンガ倉庫", "lat": 35.4521344, "lon": 139.6434272},
    {"name": "山下公園", "lat": 35.4459047, "lon": 139.6498463},
    {"name": "ランドマークタワー", "lat": 35.4546782, "lon": 139.6316336},
    {"name": "コスモワールド", "lat": 35.456442, "lon": 139.6356693},
    {"name": "野毛山動物園", "lat": 35.4478343, "lon": 139.6222125},
    {"name": "大さん橋", "lat": 35.4514568, "lon": 139.6480768},
]

# --- スポットから最寄り駅を探す関数 ---
def find_nearest_station(lat, lon):
    min_dist = float('inf')
    nearest_station_cd = None
    for station_cd, pos in station_pos.items():
        dist = ((lat - pos['lat'])**2 + (lon - pos['lon'])**2) ** 0.5
        if dist < min_dist:
            min_dist = dist
            nearest_station_cd = station_cd
    return nearest_station_cd

# --- スポットごとに最寄り駅を特定 ---
spot_to_station = {}
for spot in spots:
    nearest_cd = find_nearest_station(spot['lat'], spot['lon'])
    if nearest_cd:
        spot_to_station[spot['name']] = nearest_cd

# --- 経路探索（最寄り駅間を順に） ---
# スポットの順で最寄り駅のルートを描く（例：スポットA → B → C → ...）
all_paths = []
spot_names = list(spot_to_station.keys())
for i in range(len(spot_names) - 1):
    start = spot_to_station[spot_names[i]]
    goal = spot_to_station[spot_names[i + 1]]
    if nx.has_path(G, start, goal):
        path = nx.shortest_path(G, source=start, target=goal, weight='weight')
        all_paths.append(path)
    else:
        print(f"経路が見つかりませんでした: {spot_names[i]} → {spot_names[i + 1]}")





In [162]:
# --- Douglas-Peucker簡略化関数 ---
def simplify_line(coords, tolerance=0.0001):
    line = LineString([(lon, lat) for lat, lon in coords])  # shapelyは(x,y)=(lon,lat)
    simplified = line.simplify(tolerance, preserve_topology=False)
    return [(lat, lon) for lon, lat in simplified.coords]

In [184]:
import geopandas as gpd
from shapely.geometry import Point

def find_nearest_node(lon, lat, G_walk):
    # ノードGeoDataFrame作成（経度緯度）
    nodes_gdf = gpd.GeoDataFrame(
        {'node': list(G_walk.nodes)},
        geometry=[Point(data['x'], data['y']) for _, data in G_walk.nodes(data=True)],
        crs='EPSG:4326'
    )
    
    # 探したいポイントをGeoDataFrameで作成
    pt = gpd.GeoDataFrame(geometry=[Point(lon, lat)], crs='EPSG:4326')

    # 投影座標系に変換（距離計算用）
    nodes_proj = nodes_gdf.to_crs(epsg=3857)
    pt_proj = pt.to_crs(epsg=3857)

    # 距離計算
    nodes_proj['dist'] = nodes_proj.geometry.distance(pt_proj.geometry.iloc[0])
    nearest_node = nodes_proj.loc[nodes_proj['dist'].idxmin(), 'node']
    
    return nearest_node


def add_walking_route_to_map(m, start_lat, start_lon, goal_lat, goal_lon):
    start_node = find_nearest_walk_node(start_lat, start_lon)
    goal_node = find_nearest_walk_node(goal_lat, goal_lon)

    if nx.has_path(G_walk, start_node, goal_node):
        walk_path = nx.shortest_path(G_walk, start_node, goal_node, weight='length')
        route_coords = [(G_walk.nodes[n]['y'], G_walk.nodes[n]['x']) for n in walk_path]

        # ジオメトリ簡略化
        route_coords = simplify_line(route_coords, tolerance=0.00075)

        

        folium.PolyLine(
            route_coords,
            color='lime',
            weight=6,
            opacity=1.0,
            tooltip='最寄り駅への徒歩ルート'
        ).add_to(m)
    else:
        print(f"徒歩ルートが見つかりませんでした。")



In [185]:
# --- スポット間の駅経路 ---
spot_names = list(spot_to_station.keys())
all_paths = []
for i in range(len(spot_names) - 1):
    start = spot_to_station[spot_names[i]]
    goal = spot_to_station[spot_names[i + 1]]
    if nx.has_path(G, start, goal):
        path = nx.shortest_path(G, start, goal, weight='weight')
        all_paths.append(path)
    else:
        print(f"経路が見つかりませんでした: {spot_names[i]} → {spot_names[i+1]}")


In [186]:
from shapely.geometry import Point
import geopandas as gpd
from shapely.ops import unary_union

In [201]:
import geopandas as gpd
from shapely.geometry import Point

# --- 道路をGeoDataFrame化 ---
gdf_drive = ox.graph_to_gdfs(G_drive, nodes=False)


# --- 主要道路フィルター ---
target_highways = ['motorway']
gdf_drive = gdf_drive[
    gdf_drive['highway'].apply(lambda x: any(h in target_highways for h in (x if isinstance(x, list) else [x])))
]

# --- 道路ジオメトリの重複・重なりをまとめる ---
merged_geom = unary_union(gdf_drive.geometry)

if merged_geom.geom_type == 'MultiLineString':
    geoms = list(merged_geom.geoms)
else:
    geoms = [merged_geom]

gdf_drive_unique = gpd.GeoDataFrame(geometry=geoms, crs=gdf_drive.crs)

# --- 駅のGeoDataFrame作成 ---
station_gdf = gpd.GeoDataFrame(
    geometry=[Point(station_pos[cd]['lon'], station_pos[cd]['lat']) for cd in used_station_cds],
    crs='EPSG:4326'
).to_crs(epsg=3857)

# --- スポットのGeoDataFrame作成 ---
spot_gdf = gpd.GeoDataFrame(
    geometry=[Point(spot['lon'], spot['lat']) for spot in spots],
    crs='EPSG:4326'
).to_crs(epsg=3857)

# --- 道路GeoDataFrameも投影変換 ---
gdf_drive_proj = gdf_drive_unique.to_crs(epsg=3857)

# --- 駅とスポットのバッファ作成（500m） ---
station_buffer = station_gdf.buffer(500)
spot_buffer = spot_gdf.buffer(500)

# --- バッファ範囲を合成 ---
buffer_union = station_buffer.unary_union.union(spot_buffer.unary_union)

# --- バッファ範囲内の道路だけ抽出 ---
gdf_drive_filtered = gdf_drive_proj[gdf_drive_proj.intersects(buffer_union)]

# --- 元の座標系に戻す ---
gdf_drive_filtered = gdf_drive_filtered.to_crs(epsg=4326)


/tmp/ipykernel_1399524/2350602207.py:44: DeprecationWarning:

The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.



In [202]:


# --- 地図作成 ---
m = folium.Map(location=[35.44, 139.64], zoom_start=14, tiles='cartodbpositron')


for _, row in gdf_drive_filtered.iterrows():
    coords = [(lat, lon) for lon, lat in row['geometry'].coords]
    folium.PolyLine(coords, color='gray', weight=3, opacity=0.7).add_to(m)


# --- 鉄道路線（駅間直線）を描画 ---
for path in all_paths:
    for i in range(len(path) - 1):
        a = station_pos[path[i]]
        b = station_pos[path[i + 1]]
        folium.PolyLine(
            locations=[(a['lat'], a['lon']), (b['lat'], b['lon'])],
            color='red',
            weight=5,
            tooltip='鉄道路線'
        ).add_to(m)

# --- 観光スポットマーカー ---
for spot in spots:
    folium.Marker(
        location=(spot['lat'], spot['lon']),
        popup=spot['name'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

# --- 駅マーカー（経路内） ---
used_station_cds = set(cd for path in all_paths for cd in path)
for station_cd in used_station_cds:
    data = station_pos[station_cd]
    name = station_yokohama.loc[station_yokohama['station_cd'] == station_cd, 'station_name'].values[0]
    folium.Marker(
        location=(data['lat'], data['lon']),
        popup=name,
        icon=folium.Icon(color='red', icon='train', prefix='fa')
    ).add_to(m)

# --- 駅→スポット徒歩ルートを描画 ---
for spot in spots:
    spot_lat, spot_lon = spot['lat'], spot['lon']
    station_cd = spot_to_station[spot['name']]
    station_lat = station_pos[station_cd]['lat']
    station_lon = station_pos[station_cd]['lon']
    add_walking_route_to_map(m, station_lat, station_lon, spot_lat, spot_lon)

# --- 最寄り駅マーカー（オレンジ星） ---
for spot in spots:
    station_cd = spot_to_station[spot['name']]
    data = station_pos[station_cd]
    name = station_yokohama.loc[station_yokohama['station_cd'] == station_cd, 'station_name'].values[0]
    folium.Marker(
        location=(data['lat'], data['lon']),
        popup=f"最寄り駅: {name}",
        icon=folium.Icon(color='orange', icon='star', prefix='fa')
    ).add_to(m)

# --- 地図表示 ---
m

In [75]:
import folium

# --- 地図初期化 ---
m = folium.Map(location=[35.4437, 139.6380], zoom_start=13, tiles="CartoDB positron")

# --- 駅マーカー（ルートに含まれる駅だけ） ---
used_station_cds = set(cd for path in all_paths for cd in path)
for station_cd in used_station_cds:
    data = station_pos[station_cd]
    name = station_yokohama.loc[station_yokohama['station_cd'] == station_cd, 'station_name'].values[0]
    folium.Marker(
        location=(data['lat'], data['lon']),
        popup=name,
        icon=folium.Icon(color='red', icon='train', prefix='fa')
    ).add_to(m)

# --- 鉄道ルート（赤色の直線） ---
for path in all_paths:
    for i in range(len(path) - 1):
        a = station_pos[path[i]]
        b = station_pos[path[i + 1]]
        folium.PolyLine(
            locations=[(a['lat'], a['lon']), (b['lat'], b['lon'])],
            color='red',
            weight=4,
            tooltip='鉄道ルート（直線）'
        ).add_to(m)

# --- スポットマーカー ---
for spot in spots:
    folium.Marker(
        location=(spot['lat'], spot['lon']),
        popup=spot['name'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

# --- 駅→スポット間の徒歩ルートを直線で描画（緑色） ---
for spot in spots:
    spot_lat, spot_lon = spot['lat'], spot['lon']
    station_cd = spot_to_station[spot['name']]
    station_lat = station_pos[station_cd]['lat']
    station_lon = station_pos[station_cd]['lon']

    folium.PolyLine(
        locations=[(station_lat, station_lon), (spot_lat, spot_lon)],
        color='green',
        weight=3,
        opacity=0.8,
        dash_array='5,5',
        tooltip='徒歩ルート（直線）'
    ).add_to(m)

# --- スポットの最寄り駅をオレンジマーカーで強調 ---
for spot in spots:
    station_cd = spot_to_station[spot['name']]
    data = station_pos[station_cd]
    name = station_yokohama.loc[station_yokohama['station_cd'] == station_cd, 'station_name'].values[0]

    folium.Marker(
        location=(data['lat'], data['lon']),
        popup=f"最寄り駅: {name}",
        icon=folium.Icon(color='orange', icon='star', prefix='fa')
    ).add_to(m)


m


RE

In [76]:
import folium
import osmnx as ox
import networkx as nx
import pandas as pd
from geopy.distance import geodesic


In [77]:
# --- 観光スポットデータ ---
spots = [
    {"name": "横浜中華街", "lat": 35.4432993, "lon": 139.6456246},
    {"name": "みなとみらい21", "lat": 35.4553642, "lon": 139.6296088},
    {"name": "赤レンガ倉庫", "lat": 35.4521344, "lon": 139.6434272},
    {"name": "山下公園", "lat": 35.4459047, "lon": 139.6498463},
    {"name": "ランドマークタワー", "lat": 35.4546782, "lon": 139.6316336},
    {"name": "コスモワールド", "lat": 35.456442, "lon": 139.6356693},
    {"name": "野毛山動物園", "lat": 35.4478343, "lon": 139.6222125},
    {"name": "大さん橋", "lat": 35.4514568, "lon": 139.6480768},
]

# --- 最寄り駅を探す関数（事前定義：station_pos辞書） ---
def find_nearest_station(lat, lon):
    min_dist = float('inf')
    nearest_station_cd = None
    for station_cd, pos in station_pos.items():
        dist = ((lat - pos['lat'])**2 + (lon - pos['lon'])**2) ** 0.5
        if dist < min_dist:
            min_dist = dist
            nearest_station_cd = station_cd
    return nearest_station_cd

# --- 徒歩ノード検索 ---
def find_nearest_walk_node(lat, lon):
    return ox.distance.nearest_nodes(G_walk, lon, lat)

# --- 徒歩ルート描画関数 ---
def add_walking_route_to_map(m, start_lat, start_lon, goal_lat, goal_lon):
    start_node = find_nearest_walk_node(start_lat, start_lon)
    goal_node = find_nearest_walk_node(goal_lat, goal_lon)
    if nx.has_path(G_walk, start_node, goal_node):
        walk_path = nx.shortest_path(G_walk, start_node, goal_node, weight='length')
        route_coords = [(G_walk.nodes[n]['y'], G_walk.nodes[n]['x']) for n in walk_path]
        folium.PolyLine(route_coords, color='green', weight=4, opacity=0.7, tooltip='徒歩ルート').add_to(m)
    else:
        print(f"徒歩ルートが見つかりませんでした。")

# --- 距離しきい値に基づき徒歩or鉄道を判定 ---
def should_walk_between_spots(spot1, spot2, threshold_km=1.0):
    p1 = (spot1['lat'], spot1['lon'])
    p2 = (spot2['lat'], spot2['lon'])
    distance = geodesic(p1, p2).km
    return distance <= threshold_km

# --- 地図初期化 ---
m = folium.Map(location=[35.4437, 139.6380], zoom_start=13)

# --- 経路処理開始 ---
all_paths = []
used_station_cds = set()

for i in range(len(spots) - 1):
    spot1 = spots[i]
    spot2 = spots[i + 1]

    if should_walk_between_spots(spot1, spot2, threshold_km=1.0):
        # 徒歩でスポット同士を直接つなぐ
        add_walking_route_to_map(m, spot1['lat'], spot1['lon'], spot2['lat'], spot2['lon'])
    else:
        # 遠い場合は、スポット→駅、駅→駅、駅→スポットを描画

        # 1. スポット1 → 最寄り駅（徒歩）
        start_cd = find_nearest_station(spot1['lat'], spot1['lon'])
        if start_cd:
            start_station = station_pos[start_cd]
            add_walking_route_to_map(m, spot1['lat'], spot1['lon'], start_station['lat'], start_station['lon'])
        
        # 2. スポット2 → 最寄り駅（徒歩）
        goal_cd = find_nearest_station(spot2['lat'], spot2['lon'])
        if goal_cd:
            goal_station = station_pos[goal_cd]
            add_walking_route_to_map(m, spot2['lat'], spot2['lon'], goal_station['lat'], goal_station['lon'])

        # 3. 駅間移動（鉄道）
        if start_cd and goal_cd and nx.has_path(G, start_cd, goal_cd):
            path = nx.shortest_path(G, source=start_cd, target=goal_cd, weight='weight')
            all_paths.append(path)
            used_station_cds.update(path)
        else:
            print(f"駅経由ルートが見つかりません: {spot1['name']} → {spot2['name']}")


# --- 駅マーカー（赤） ---
for station_cd in used_station_cds:
    data = station_pos[station_cd]
    name = station_yokohama.loc[station_yokohama['station_cd'] == station_cd, 'station_name'].values[0]
    folium.Marker(
        location=(data['lat'], data['lon']),
        popup=name,
        icon=folium.Icon(color='red', icon='train', prefix='fa')
    ).add_to(m)

# --- 駅経路描画（赤線） ---
for path in all_paths:
    for i in range(len(path) - 1):
        a = station_pos[path[i]]
        b = station_pos[path[i + 1]]
        folium.PolyLine(
            locations=[(a['lat'], a['lon']), (b['lat'], b['lon'])],
            color='red', weight=5, tooltip='駅経路'
        ).add_to(m)

# --- スポットマーカー（青） ---
for spot in spots:
    folium.Marker(
        location=(spot['lat'], spot['lon']),
        popup=spot['name'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

m.save("yokohama_map.html")


徒歩でのスポット最短距離ルート

In [78]:
G_undirected = G_walk.to_undirected()


In [79]:
# 各スポットの最寄りノード（GのノードID）を取得
spot_nodes = []
for spot in spots:
    node = ox.distance.nearest_nodes(G_undirected, spot["lon"], spot["lat"])
    spot_nodes.append((spot["name"], node))


In [80]:
# スポットの順に経路をつなぐ
routes = []
for i in range(len(spot_nodes) - 1):
    start_node = spot_nodes[i][1]
    end_node = spot_nodes[i + 1][1]
    if nx.has_path(G_walk, start_node, end_node):
        route = nx.shortest_path(G_undirected, start_node, end_node, weight='length')
        routes.append(route)


In [81]:
# 初期マップ
m = folium.Map(location=[35.4437, 139.6380], zoom_start=14)

# 各スポットにマーカー追加
for spot in spots:
    folium.Marker(
        location=(spot['lat'], spot['lon']),
        popup=spot['name'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

# 経路（緑線）を描画
for route in routes:
    route_coords = [(G_undirected.nodes[n]['y'], G_undirected.nodes[n]['x']) for n in route]
    folium.PolyLine(route_coords, color='green', weight=4, opacity=0.8, tooltip='徒歩経路').add_to(m)

# 表示または保存
#m.save("yokohama_route_map.html")
m


In [82]:
import osmnx as ox
import networkx as nx

# スポットデータ（緯度・経度）
spots = [
    {"name": "横浜中華街", "lat": 35.4432993, "lon": 139.6456246},
    {"name": "みなとみらい21", "lat": 35.4553642, "lon": 139.6296088},
    {"name": "赤レンガ倉庫", "lat": 35.4521344, "lon": 139.6434272},
    {"name": "山下公園", "lat": 35.4459047, "lon": 139.6498463},
    {"name": "ランドマークタワー", "lat": 35.4546782, "lon": 139.6316336},
    {"name": "コスモワールド", "lat": 35.456442, "lon": 139.6356693},
    {"name": "野毛山動物園", "lat": 35.4478343, "lon": 139.6222125},
    {"name": "大さん橋", "lat": 35.4514568, "lon": 139.6480768},
]

# 中心と範囲でOSMグラフ（徒歩用）を取得
center_lat = sum(spot["lat"] for spot in spots) / len(spots)
center_lon = sum(spot["lon"] for spot in spots) / len(spots)
G = ox.graph_from_point((center_lat, center_lon), dist=1500, network_type='walk', simplify=True)

# ノードを最寄りのOSMノードにスナップ
for spot in spots:
    spot["node"] = ox.nearest_nodes(G, X=spot["lon"], Y=spot["lat"])

# 最短経路を全ペアで探索して、利用されたエッジを記録
edge_usage = {}
for i in range(len(spots)):
    for j in range(i + 1, len(spots)):
        source = spots[i]["node"]
        target = spots[j]["node"]
        try:
            path = nx.shortest_path(G, source=source, target=target, weight="length")
            path_edges = list(zip(path[:-1], path[1:]))
            for edge in path_edges:
                edge_usage[edge] = edge_usage.get(edge, 0) + 1
        except nx.NetworkXNoPath:
            continue

# 媒介中心性をNetworkXで計算（全ノードから）
betweenness = nx.edge_betweenness_centrality(G, weight='length')

# 結果（中心性の高いエッジ上位表示）
top_edges = sorted(betweenness.items(), key=lambda x: -x[1])[:10]
for (u, v), score in top_edges:
    print(f"Edge ({u} -> {v}): Betweenness Centrality = {score:.5f}")


KeyboardInterrupt: 